<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/banking77_English%20-%20ConvBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jun 28 15:56:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!pip install transformers
!pip install datasets

In [5]:
import os
import gc
import numpy as np
import pandas as pd

import torch
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import f1_score, accuracy_score

from transformers import ConvBertForSequenceClassification, ConvBertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset

device = 'cuda'

In [4]:
dataset = load_dataset('banking77')

Using custom data configuration default


Dataset banking77 downloaded and prepared to /root/.cache/huggingface/datasets/banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b. Subsequent calls will reuse this data.


# Tokenize data

In [11]:
def get_banking77_tokens(data, label, maxlen):
    ids, labels = [], []
    for text, label in zip(data, label):
        tokens = tokenizer.encode_plus(
            text=text,
            truncation=True, 
            max_length=maxlen,
            padding='max_length',
            return_tensors='pt'
        )
        labels.append(int(label))
        ids.append(tokens['input_ids'])

    ids = torch.vstack(ids)
    return ids, torch.tensor(labels)

In [12]:
# artigo: https://arxiv.org/pdf/2003.04807.pdf

BSIZE = 16
MAX_LEN = 32
path_model = 'YituTech/conv-bert-base'
tokenizer = ConvBertTokenizer.from_pretrained(path_model)

# Train
train_texts, train_labels = get_banking77_tokens(dataset['train']['text'], dataset['train']['label'], MAX_LEN)
ds_train = TensorDataset(train_texts, train_labels)

# Test
test_texts, test_labels = get_banking77_tokens(dataset['test']['text'], dataset['test']['label'], MAX_LEN)
ds_test = TensorDataset(test_texts, test_labels)

# Dataloaders
dataloader = {
    'train': DataLoader(
        dataset=ds_train,
        batch_size=BSIZE,
        shuffle=True,
        pin_memory=True,
        num_workers=os.cpu_count()
         ),
    'test': DataLoader(
        dataset=ds_test,
        batch_size=BSIZE,
        shuffle=True,
        pin_memory=True,
        num_workers=os.cpu_count()
         )
}

# - - - - -
_ = {_:len(dataloader[_]) for _ in dataloader.keys()}
print(_)
ids_batch, label_batch = next(iter(dataloader['train']))
ids_batch[0], label_batch[0]

{'train': 626, 'test': 193}


(tensor([  101,  2129,  2079,  1045,  2131,  1037,  4487, 13102,  8820,  3468,
          7484,  4003,  2004,  2092,  1029,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]), tensor(37))

In [16]:
model = ConvBertForSequenceClassification.from_pretrained(
    path_model, 
    num_labels=len(list(set(dataset['test']['label']))), 
    return_dict=True
    )

# - - - - -
outs = model(input_ids=ids_batch, labels=label_batch)
outs.keys()

Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


odict_keys(['loss', 'logits'])

In [18]:
try:
    del model
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

def train(model, loader, optim, scheduler):
    total_acc, total_f1, total_loss = 0,0,0
    model.train()
    for batch in loader:
        model.zero_grad()
        
        batch_ids, batch_label = (b.to('cuda') for b in batch)
        
        outs = model(input_ids=batch_ids, labels=batch_label)
        
        loss = outs['loss']
        total_loss += loss.item()
        
        loss.backward()
        optim.step()
        scheduler.step()
        
        logits = torch.nn.functional.softmax(outs['logits'].cpu().detach(), dim=-1).numpy()
        y_pred = np.argmax(logits, axis=1)
        total_f1  += f1_score(y_true=batch_label.cpu(), y_pred=y_pred, average='macro')
        total_acc += accuracy_score(y_true=batch_label.cpu(), y_pred=y_pred)
    
    avg_loss = total_loss/len(loader)
    avg_f1 = total_f1/len(loader)
    avg_acc = total_acc/len(loader)

    return avg_loss, avg_f1, avg_acc

def evaluate(model, loader):
    total_acc, total_f1, total_loss = 0,0,0
    model.eval()
    for batch in loader:
        model.zero_grad()
        
        batch_ids, batch_label = (b.to('cuda') for b in batch)
        
        with torch.no_grad():
            outs = model(input_ids=batch_ids, labels=batch_label)
        
        loss = outs['loss']
        total_loss += loss.item()
        
        logits = torch.nn.functional.softmax(outs['logits'].cpu().detach(), dim=-1).numpy()
        y_pred = np.argmax(logits, axis=1)
        total_f1  += f1_score(y_true=batch_label.cpu(), y_pred=y_pred, average='macro')
        total_acc += accuracy_score(y_true=batch_label.cpu(), y_pred=y_pred)
    
    avg_loss = total_loss/len(loader)
    avg_f1 = total_f1/len(loader)
    avg_acc = total_acc/len(loader)

    return avg_loss, avg_f1, avg_acc

# - - - - -
model = ConvBertForSequenceClassification.from_pretrained(
    path_model, 
    num_labels=len(list(set(dataset['test']['label']))), 
    return_dict=True
    ).to(device)

optim = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

N_EPOCHS = 3

total_steps = len(dataloader['train']) * N_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optim, 
    num_warmup_steps = int(0.02 * total_steps), # 2% of warmup
    num_training_steps = total_steps
    )

training_stats = []
# ----------------------------------------------------------------------------------------
for idx in range(N_EPOCHS):
    loss_train, f1_train, acc_train = train(
        model, 
        dataloader['train'], 
        optim, 
        scheduler=scheduler
        )
    loss_eval,  f1_eval,  acc_eval = evaluate(
        model, 
        dataloader['test']
        )
    
    print(f'Epoch {idx}/{N_EPOCHS-1}:')
    print(f'\tTrain: Loss: {loss_train:<5.3} -- F1: {f1_train:<5.3} -- ACC: {acc_train:.3}')
    print(f'\tEval:  Loss: {loss_eval:<5.3} -- F1: {f1_eval:<5.3} -- ACC: {acc_eval:.3}')

    training_stats.append(
        {
            'epoch': idx,
            'Training Loss': loss_train,
            'Training Acc': acc_train,
            'Training F1': f1_train,
            'Eval Loss': loss_eval,
            'Eval Acc': acc_eval,
            'Eval F1': f1_eval,
        }
    )

Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 0/2:
	Train: Loss: 3.14  -- F1: 0.197 -- ACC: 0.286
	Eval:  Loss: 1.48  -- F1: 0.535 -- ACC: 0.684
Epoch 1/2:
	Train: Loss: 0.831 -- F1: 0.738 -- ACC: 0.834
	Eval:  Loss: 0.515 -- F1: 0.819 -- ACC: 0.893
Epoch 2/2:
	Train: Loss: 0.331 -- F1: 0.896 -- ACC: 0.94
	Eval:  Loss: 0.383 -- F1: 0.851 -- ACC: 0.913


In [19]:
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
pd.set_option('precision', 3)
df_stats

,Training Loss,Training Acc,Training F1,Eval Loss,Eval Acc,Eval F1
epoch,,,,,,
0,3.142,0.286,0.197,1.476,0.684,0.535
1,0.831,0.834,0.738,0.515,0.893,0.819
2,0.331,0.940,0.896,0.383,0.913,0.851
